In [2]:
# PROJECT: COINREASON-7B | END-TO-END PRODUCTION PIPELINE
# AUTHOR: Sarfras
# ENVIRONMENT: Kaggle T4 GPU
# DESCRIPTION: Creates data, fine-tunes Mistral-7B, validates, and deploys to Hub.

!pip install -U datasets fsspec gcsfs --no-cache-dir

import os
import json
import torch
import gc

# 1. SETUP & INSTALLATION 
# Force Single GPU mode to prevent memory crashes on free tier
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

try:
    import unsloth
except ImportError:
    # Quiet installation for Kaggle
    !pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git" --no-cache-dir
    !pip install --no-deps accelerate bitsandbytes "trl<0.9.0" --no-cache-dir
    !pip install --force-reinstall "peft>=0.7.1" --no-deps --no-cache-dir

from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

INFO: pip is looking at multiple versions of gcsfs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.4.1
    Uninstalling datasets-4.4.1:
      Successfully uninstalled datasets-4.4.1
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.0
    Uninstalling gcsfs-2025.3.0:
      Successfully uninstalled gcsfs-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
bigframes 2.26.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mgb6rf0t/unsloth_475994498

2026-01-10 07:53:51.840551: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768031632.026456      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768031632.079846      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768031632.513414      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768031632.513448      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768031632.513451      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# PHASE 1: DATA ENGINEERING (INTERNAL GENERATION)

def create_dataset_file(filename="reasoning_dataset.json"):
    # High-Quality Examples (Manually Cured for Prototype)
    gold_data = [
        {
            "text": "Bitcoin just broke $65k with massive volume! The bears are getting wrecked. 🚀 #BTC",
            "sentiment": "Bullish",
            "explanation": "The breakout above the psychological $65k level accompanied by high volume indicates strong momentum and buyer conviction.",
            "market_implication": "Expect a continuation rally towards $68k as short positions are liquidated."
        },
        {
            "text": "Whales are moving 5000 BTC to Binance. This doesn't look good. Support at 60k is weak.",
            "sentiment": "Bearish",
            "explanation": "Large inflows of BTC to exchanges (Whale movement) typically signal an intent to sell, increasing sell-side pressure.",
            "market_implication": "Price likely to test the $60k support; a breakdown could trigger a flush to $58k."
        },
        {
            "text": "RSI is extremely overbought on the 4H chart. Time to take some profits off the table.",
            "sentiment": "Bearish",
            "explanation": "An overbought RSI (>70) suggests the asset is overextended in the short term and due for a mean reversion or pullback.",
            "market_implication": "Expect a short-term correction or consolidation before any further upside."
        },
        {
            "text": "Golden Cross just happened on the daily chart! 50MA crossed above 200MA. Bull market confirmed.",
            "sentiment": "Bullish",
            "explanation": "A Golden Cross is a lagging but powerful long-term technical indicator signaling a shift from a downtrend to a sustained uptrend.",
            "market_implication": "Long-term bullish trend confirmed; dips are likely buying opportunities."
        },
        {
            "text": "Fed just announced rate hikes. Risk assets are going to bleed. Selling my bags.",
            "sentiment": "Bearish",
            "explanation": "Interest rate hikes increase the cost of borrowing and strengthen the dollar, which historically correlates with a drop in risk assets like Crypto.",
            "market_implication": "Bitcoin likely to face downward pressure as macro liquidity tightens."
        }
    ] * 20  # Duplicate to create 100 examples for the training loop

    with open(filename, "w") as f:
        json.dump(gold_data, f)
    
    return filename, len(gold_data)

filename, count = create_dataset_file()
print(f"Dataset '{filename}' created with {count} examples.")

Dataset 'reasoning_dataset.json' created with 100 examples.


In [4]:
# PHASE 2: MODEL TRAINING (MISTRAL-7B)

# 1. Load Data
dataset = load_dataset("json", data_files="reasoning_dataset.json", split="train")

def format_mistral(ex):
    return {
        "text": f"""<s>[INST] Analyze the following Bitcoin market text for sentiment and short-horizon implication.

Text: {ex['text']}

Provide output in this exact format:
Sentiment: [Bullish/Bearish]
Explanation: [reasoning]
Market Implication: [brief BTC price direction outlook][/INST]

Sentiment: {ex['sentiment']}
Explanation: {ex['explanation']}
Market Implication: {ex['market_implication']}</s>"""
    }

dataset = dataset.map(format_mistral)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# 2. Load Model
print("Loading Model Weights (4-bit Quantization)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True,
)

# 3. Apply Adapters
print("Injecting LoRA Adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

# 4. Train
print("STARTING TRAINING...")
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    dataset_text_field = "text",
    max_seq_length = 1024,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 60, # Quick training for prototype
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "crypto_mistral_outputs",
        report_to = "none",
    ),
)

trainer_stats = trainer.train()
print(f"TRAINING COMPLETE. Time: {trainer_stats.metrics['train_runtime']}s")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Loading Model Weights (4-bit Quantization)...
==((====))==  Unsloth 2026.1.2: Fast Mistral patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14 | Num Epochs = 30 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.106900
10,0.883100
15,0.740500
20,0.615100
25,0.508100
30,0.426900
35,0.377500
40,0.355200
45,0.336500
50,0.327200


TRAINING COMPLETE. Time: 1683.7124s


In [5]:
# PHASE 3: VALIDATION & DEPLOYMENT

# 1. Test Inference
FastLanguageModel.for_inference(model)
test_tweet = "Bitcoin volume is dying and we are stuck at resistance. I think we go down."
prompt = f"""<s>[INST] Analyze the following Bitcoin market text for sentiment and short-horizon implication.

Text: {test_tweet}

Provide output in this exact format:
Sentiment: [Bullish/Bearish]
Explanation: [reasoning]
Market Implication: [brief BTC price direction outlook][/INST]"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True, temperature=0.1)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1].strip()

print(f"TEST INPUT: {test_tweet}")
print(f"MODEL OUTPUT:\n{answer}")

TEST INPUT: Bitcoin volume is dying and we are stuck at resistance. I think we go down.
MODEL OUTPUT:
Analyze the following Bitcoin market text for sentiment and short-horizon implication.

Text: Bitcoin volume is dying and we are stuck at resistance. I think we go down.

Provide output in this exact format:
Sentiment: [Bullish/Bearish]
Explanation: [reasoning]
Market Implication: [brief BTC price direction outlook]

Sentiment: Bearish
Explanation: High volume is typically associated with strong momentum. The lack of it suggests a lack of conviction, increasing the likelihood of a move downwards.
Market Implication: Expect a test of the support levels before any bullish reversal.


In [7]:
from huggingface_hub import notebook_login
notebook_login() 

import torch
from unsloth import FastLanguageModel

model.push_to_hub("sarfras/coinreason-7b-lora", token=True)
tokenizer.push_to_hub("sarfras/coinreason-7b-lora", token=True)
dataset.push_to_hub("sarfras/coinreason-7b-lora-dataset", token=True)
print("Done")

README.md:   0%|          | 0.00/575 [00:00<?, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Saved model to https://huggingface.co/sarfras/coinreason-7b-lora


Processing Files (0 / 0): |          |  0.00B /  0.00B            

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Done
